#This is for t1+t2 fixed and t1 - t2 varying OBC

In [ ]:
using ITensors, ITensorMPS, Plots

function run_dmrg_ssh(N::Int, t1::Float64, t2::Float64, nsweeps::Int, maxdim::Vector{Int}, cutoff::Vector{Float64})
    # Define fermionic sites
    sites = siteinds("Fermion", N)

    # Construct SSH Hamiltonian as an MPO
    os = OpSum()
    for j in 1:N-1
        hopping = j % 2 == 1 ? t1 : t2
        os += -hopping, "Cdag", j, "C", j + 1
        os += -hopping, "Cdag", j + 1, "C", j
    end



    H = MPO(os, sites)

    # Compute the ground state psi0
    state = [isodd(n) ? "1" : "0" for n in 1:N]  # Alternating filling pattern
    psi0_init = randomMPS(sites,state)
    energy_ground, psi_ground = dmrg(H, psi0_init; nsweeps, maxdim, cutoff, noise)

    # Compute the first excited state psi1 
    psi1_init = randomMPS(sites,state)  
    energy_excited, psi_excited = dmrg(H, [psi_ground], psi1_init; nsweeps, maxdim, cutoff, noise, weight=weight)

    # Compute energy gap
    energy_gap = energy_excited - energy_ground

    # Compute entanglement entropy for a subsystem
    function entanglement_entropy(psi, site)
        psi = orthogonalize(psi, site)
        U, S, V = svd(psi[site], (linkinds(psi, site-1)..., siteinds(psi, site)...))
        SvN = 0.0
        for n = 1:dim(S, 1)
            p = S[n, n]^2
            SvN -= p * log(p)
        end
        return SvN
    end

    subsystem_size = div(N, 2)  # Compute entropy at middle site
    entropy = entanglement_entropy(psi_ground, subsystem_size)




    return energy_ground, psi_ground, energy_excited, psi_excited, energy_gap, entropy
end

# Parameters
N = 200           # Number of sites
h = 4.0
weight = 2 
noise = [1E-10]
delta_t_values = collect(-0.2:0.01:0.1)  # Varying delta t from -0.5 to 0.5
nsweeps = 5      # Number of DMRG sweeps
maxdim = [20, 40, 50, 100, 200] 
cutoff = [1E-10] 

# Compute entanglement entropy for different delta t values
entropies = []
for delta_t in delta_t_values
    t1 = 1.0 + delta_t
    t2 = 1.0 - delta_t
    _, _, _, _, _, entropy = run_dmrg_ssh(N, t1, t2, nsweeps, maxdim, cutoff)
    push!(entropies, entropy)
end

# Plot EE vs delta t
plot(delta_t_values, entropies, xlabel="Δt", ylabel="Entanglement Entropy", title="EE vs Δt in SSH Model for N= 200 sites", marker=:o)
savefig("EE_vs_delta_t.png")


After sweep 1 energy=-132.59341468665875  maxlinkdim=4 maxerr=4.91E-11 time=0.218
After sweep 2 energy=-132.83849139101576  maxlinkdim=15 maxerr=9.12E-11 time=0.235
After sweep 3 energy=-132.83884429860743  maxlinkdim=29 maxerr=9.99E-11 time=0.477
After sweep 4 energy=-132.83884479966616  maxlinkdim=28 maxerr=9.97E-11 time=0.581
After sweep 5 energy=-132.8388447994841  maxlinkdim=27 maxerr=9.98E-11 time=0.557
After sweep 1 energy=-131.22034266595617  maxlinkdim=4 maxerr=5.89E-11 time=0.201
After sweep 2 energy=-132.83396143368861  maxlinkdim=16 maxerr=9.98E-11 time=0.276
After sweep 3 energy=-132.83884281032454  maxlinkdim=29 maxerr=9.98E-11 time=0.527
After sweep 4 energy=-132.83884479489376  maxlinkdim=28 maxerr=1.00E-10 time=0.664
After sweep 5 energy=-132.83884479177297  maxlinkdim=27 maxerr=1.00E-10 time=0.889
After sweep 1 energy=-132.07735220042613  maxlinkdim=4 maxerr=4.90E-11 time=0.180
After sweep 2 energy=-132.34487230586907  maxlinkdim=15 maxerr=8.16E-11 time=0.246
After sw

"/home/photon/Downloads/Cardy_Entropy/EE_vs_delta_t.png"

#This is for S(l)=C/6 log((L\pi)sin(pi.l/L))

In [55]:
using ITensors, ITensorMPS, Plots, LsqFit

function run_dmrg_ssh(N::Int, t1::Float64, t2::Float64, nsweeps::Int, maxdim::Vector{Int}, cutoff::Vector{Float64})
    # Define fermionic sites
    sites = siteinds("Fermion", N)

    # Construct SSH Hamiltonian as an MPO
    os = OpSum()
    for j in 1:N-1
        hopping = j % 2 == 1 ? t1 : t2
        os += -hopping, "Cdag", j, "C", j + 1
        os += -hopping, "Cdag", j + 1, "C", j
    end

    H = MPO(os, sites)

    # Compute the ground state psi0
    state = [isodd(n) ? "1" : "0" for n in 1:N]  # Alternating filling pattern
    psi0_init = randomMPS(sites, state)
    energy_ground, psi_ground = dmrg(H, psi0_init; nsweeps, maxdim, cutoff, noise)

    # Compute entanglement entropy for a subsystem
    function entanglement_entropy(psi, site)
        psi = orthogonalize(psi, site)
        U, S, V = svd(psi[site], (linkinds(psi, site-1)..., siteinds(psi, site)...))
        SvN = 0.0
        for n = 1:dim(S, 1)
            p = S[n, n]^2
            if p > 0
                SvN -= p * log(p)
            end
        end
        return SvN
    end

    return psi_ground, entanglement_entropy
end

# Parameters
N = 2000         # Number of sites
h = 4.0
weight = 2 
noise = [1E-10]
delta_t = 0.0  # Set delta_t to zero
nsweeps = 5      # Number of DMRG sweeps
maxdim = [20, 40, 50, 100, 200] 
cutoff = [1E-10] 

# Compute entanglement entropy for delta t = 0
t1 = 1.00 + delta_t
t2 = 1.00 - delta_t
psi_ground, entanglement_entropy = run_dmrg_ssh(N, t1, t2, nsweeps, maxdim, cutoff)
l_values = [l for l in 2:2:div(N,2)] 

entropy_values = [entanglement_entropy(psi_ground, l) for l in l_values]

# Fit the function a(l) = (c/6) * log((L/pi) * sin(pi*l/L))
L = N
define_model(l, c) = (c/6) .* log.((L/pi) .* sin.(pi .* l ./ L))

fit = curve_fit(define_model, l_values, entropy_values, [1.0])
c_value = fit.param[1]
println("Value of c for delta_t = 0: ", c_value)

# Plot EE vs l
plot(l_values, entropy_values, xlabel="l", ylabel="EE", title="EE vs l for N=2000 sites ", marker=:o, label="EE(l)")
savefig("EE_vs_l_delta_t_0.png")


After sweep 1 energy=-1263.0871302055377  maxlinkdim=4 maxerr=4.17E-11 time=2.605
After sweep 2 energy=-1272.2415919757643  maxlinkdim=16 maxerr=6.76E-11 time=3.468
After sweep 3 energy=-1272.7488819278753  maxlinkdim=50 maxerr=1.54E-10 time=11.838
After sweep 4 energy=-1272.8094109965282  maxlinkdim=79 maxerr=1.00E-10 time=28.965
After sweep 5 energy=-1272.8315895732305  maxlinkdim=119 maxerr=1.00E-10 time=30.344
Value of c for delta_t = 0: 1.0899212774453337


"/home/photon/Downloads/Cardy_Entropy/EE_vs_l_delta_t_0.png"

In [35]:
using ITensors, ITensorMPS, Plots, LsqFit

function dmrg_ssh(N::Int, t1::Float64, t2::Float64, nsweeps::Int, maxdim::Vector{Int}, cutoff::Vector{Float64})
    sites = siteinds("Fermion", N)
    os = OpSum()
    
    for j in 1:N-1
        hopping = isodd(j) ? t1 : t2
        os += -hopping, "Cdag", j, "C", j+1
        os += -hopping, "Cdag", j+1, "C", j
    end

    H = MPO(os, sites)

    state = [isodd(n) ? "1" : "0" for n in 1:N]
    psi0 = randomMPS(sites, state)



    # Run DMRG
    energy, psi = dmrg(H, psi0; nsweeps)

    # Function to compute entanglement entropy
    function entanglement_entropy(psi, site)
        psi = orthogonalize(psi, site)
        U, S, V = svd(psi[site], (linkinds(psi, site-1)..., siteinds(psi, site)...))
        SvN = 0.0
        for i = 1:dim(S, 1)
            p = S[i, i]^2
            if p > 0
                SvN -= p * log(p)
            end
        end
        return SvN
    end

    return psi, entanglement_entropy
end

# Parameters
N = 200
δt = 0.0
nsweeps = 5
maxdim = [20, 40, 50, 100, 200]
cutoff = [1E-10]

t1 = 1.0 + δt
t2 = 1.2 - δt

# Run DMRG
psi, entanglement_entropy = dmrg_ssh(N, t1, t2, nsweeps, maxdim, cutoff)

# Compute entanglement entropy values
l_values = [l for l in 2:2:div(N,2)] 

entropy_val = [entanglement_entropy(psi, l) for l in l_values]
# Define model for curve fitting
L = N
define_model(l, c) = (c/ 6) .* log.((L / π) .* sin.(π .* l ./ L))

# Fit the central charge
fit = curve_fit(define_model, l_values, entropy_val, [1.0])
c_value = fit.param[1]
println("Value of c: ", c_value)

# Plot results
plot(l_values, entropy_val, xlabel="l", ylabel="EE", label="Data")
savefig("EE_vs_l.png")


After sweep 1 energy=-140.54437110320782  maxlinkdim=4 maxerr=0.00E+00 time=0.139
After sweep 2 energy=-141.2257784665416  maxlinkdim=16 maxerr=0.00E+00 time=0.180
After sweep 3 energy=-141.23045018428496  maxlinkdim=64 maxerr=2.21E-16 time=0.818
After sweep 4 energy=-141.23045971472578  maxlinkdim=131 maxerr=2.22E-16 time=5.320
After sweep 5 energy=-141.23046024193968  maxlinkdim=176 maxerr=2.22E-16 time=8.422
Value of c: 1.2439834928543088


"/home/photon/Downloads/Cardy_Entropy/EE_vs_l.png"